# Deployment
> run this notebook during deployment

To check time offset correction use [https://www.utctime.net](https://www.utctime.net)

> Data from the RTE website collected from 0300 - 0400 Z is averaged and attributed to the time 0300Z. So when you submit a forecast between 0350-0410 Z , it is really meant for a T+0 = 0300Z. Finally, 0300Z + 18 hours = 2100Z as given in the response.

USE `Energy[t-1, t-2,...]`, and `Forecast[t+17]` , i.e. `Energy[t-1]` is current reading!

our password is` 961993551 `
```python
Y_t18_prediction = 3500
url_to_submit = f'http://3.1.52.222/submit/pred?pwd=961993551&value={Y_t18_prediction}'

with urllib.request.urlopen(url_to_submit) as response:
   html_valid = response.read()
```
then convert to pyhton str
```python
str(html_valid.decode())
```
if valid submission, we will receive:
```
> 'Submitted \nReceived: 20-Jul-2020 04:02:50 UTC \nFor     : 20-Jul-2020 21:00:00 UTC \nValue   : 3500 \n'
```
and for wrong timing/passwords:
```
if password is wrong:
> 'Password invalid \n'
if submission is too early or late:
> 'Submission time closed \n'
```

an example for wrong submission code
```python
html=b''
try:
    with urllib.request.urlopen('http://3.1.52.222/submit/pred?pwd=961993551&value=3500') as response:
        html = response.read()
except urllib.request.HTTPError:
    html = b'server down'

print(html.decode())
```

`> server down`<br>
or<br>
```python
html.decode().split('\n')
```
`> ['Password invalid ', '']`

In [363]:
# # Set to FALSE when deploying!!! # #
TESTING = True
# # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # #

%load_ext autoreload
%autoreload 2

import os, glob, pickle
import urllib.request
import torch, numpy as np, matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
#     device = torch.device('cuda:3')
else:
    device = torch.device("cpu")
print(f'Available device: {device}')

import os, pandas as pd, numpy as np
import datetime

#For running in JupyterHub:
if os.path.basename(os.getcwd())!='P003':
    print('Not in /P003 folder, changing directory to P003')
    lib_path = os.path.expanduser(os.path.relpath('~/images/codesDIR/datathon2020/P003'))
    os.chdir(lib_path)

# Import src.datautils
    
from src import datautils

import matplotlib
plt.style.use('ggplot')
%matplotlib inline


# my servers offset time:
time_offset = pd.Timedelta(minutes=6,seconds=-9) # correction to cpu time
if TESTING:
    time_offset = pd.Timedelta(hours=10,minutes=6,seconds=-9)

def utc_now(time_delta = time_offset):
    t = datetime.datetime.utcnow()
    utc_time = pd.to_datetime('{}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}'.format(
        t.year,t.month,t.day, t.hour,t.minute,t.second))+time_delta
    return utc_time

def need_time():
    '''Current time needed for Energy kWh readings'''
    curr_time_utc = utc_now()
    curr_minutes = curr_time_utc.minute
    
    if (curr_minutes<10) or (curr_minutes>29):
        return curr_time_utc.round('H')-pd.Timedelta(hours=1)
    return curr_time_utc.round('H')


def send_value2url(val):
    html=b''
    passwrd = '961993551'
    url_add = f'http://3.1.52.222/submit/pred?pwd={passwrd}&value={val}'
    try:
        with urllib.request.urlopen(url_add) as response:
            html = response.read()
    except urllib.request.HTTPError:
        html = b'server is down'
        print(utc_now(),f' (for {need_time()}):',html.decode(),url_add)
    return html


def read_wind_forecasts_w_range(wind_speed_range):
    dfs = []
    for model_n in range(1,3,1):
        for farm_k in range(len(datautils.locations)):
            dfs.append(
                datautils.readlocation_as_vec( model_n,farm_k, wind_speed_range).interpolate(
                    method='linear').reindex(wind_speed_range) )
    # concatenate along axis 1 using datetime as reference
    # then take average of two forecasts
    return datautils.average_forecast_models( pd.concat(dfs,axis=1) )


# CONSTANTS:
window_size = 40
lead_time = 18

# Normalisation:
# ENERGY :
shift_ = 18000.0 # e.g. mean , lookup from stats, you can round it up
scale_ = 40000.0 # e.g. 2xS.D., lookup from stats
# x_norm = (x-shift_)/scale_

# WIND SPEED:
wind_scale_ = 8.0 # half of max speed (wind vector)




# location of networks
model_dir = os.path.relpath('./best_models')
# get all models that match the pattern:
model_filenames = glob.glob(os.path.join(model_dir,'20Jul2020_1155_ver*'))

modellist = []
for fname in model_filenames:
    with open(fname, 'rb') as f:
        print(f'Loading {fname}')
        modellist.append(pickle.load(f))

for net in modellist:
    net.to(device)

def predictT18(x,y_0,nets=modellist):
    results = []
    y_0 = torch.from_numpy(y_0.astype(np.float32)).to(device)
    x = torch.from_numpy(x.astype(np.float32)).to(device)
    with torch.no_grad():
        for net in modellist:
            net.eval()
            results.append(net(x,y_0).detach().cpu().numpy())
    return results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Available device: cuda
Loading best_models/20Jul2020_1155_ver2.pkl
Loading best_models/20Jul2020_1155_ver1.pkl
Loading best_models/20Jul2020_1155_ver5.pkl
Loading best_models/20Jul2020_1155_ver3.pkl
Loading best_models/20Jul2020_1155_ver4.pkl


In [376]:
print('\n'+'- - '*5+'\nDeployment >'+
      f'{"Testing" if TESTING else " ON  <"}\n'+
      '- - '*5+'\n')
# Deployment loop:

t0 = utc_now()
# Download New Forecasts:
datautils.download_forecasts_all()

# we need this range of dates for features
oldest_time = need_time()-pd.Timedelta(hours=window_size-1)
wind_speed_latest_time = need_time()+pd.Timedelta(hours=18)
wind_speed_range = pd.date_range(oldest_time,wind_speed_latest_time,freq='H')

wind_df = read_wind_forecasts_w_range(wind_speed_range)
print(f'---\nTime elapsed: {utc_now()-t0}\n---\n')

wind_norm = wind_df.values/wind_scale_
wind_speeds = np.sqrt(wind_norm[:,0::2]**2 + wind_norm[:,1::2]**2)# wind speed from wind vectors

X_norm_diffwindows = []
for l in range(wind_speeds.shape[1]):
    X_norm_diffwindows.append(
        datautils.windowed_diff_data(wind_speeds[:,l], lead_time=lead_time, window_size=window_size))
# print for debugging
# print('\nX(t+18),X(T+0),X(T+0)-X(T-1),...,X(T-window_size+2)-X(T-window_size+1):\n',
#       [l.shape for l in X_norm_diffwindows])

t0 = utc_now()
# AI4Impact source:
datautils.download_energy_latest()
energy_1 = datautils.read_ai4impact_energy('datasets/energy-ile-de-france.csv')

# RTE source:
RTE_file_path = datautils.download_raw_from_RTE('real_time',return_filelist=True)
energy_2=datautils.read_RTE_as_kwh(RTE_file_path[0],convert2UTC=True)

print(f'---\nTime elapsed: {utc_now()-t0}\n---\n')

# Select latest energy source
enrg_src = np.argmax([energy_1.index[-1], energy_2.index[-1]])
energy_df  = energy_1 if enrg_src==0 else energy_2
energy_date_range = pd.date_range(energy_df.index[0],energy_df.index[-1],freq='H')


need_range = pd.date_range(need_time()-pd.Timedelta(hours=window_size-1),need_time(),freq='H')


if energy_df[energy_df.index==need_time()].values.shape[0]==0:
    Y0=None
    Y=None
else:
    Y = (energy_df.loc[need_range].values[::-1,0].reshape(1,-1) - shift_)/scale_
    Y0 = Y[:,:1]
    Ydiff = Y[:,0:-1] - Y[:,1:]
    Xdeploy = [Ydiff]
    Xdeploy.extend(X_norm_diffwindows)
    Xdeploy = np.concatenate(Xdeploy, axis=1)
    # Predict using 5 best models, and de-normalise, take mean for all predictions:
    Y_pred = np.mean(np.array(predictT18(Xdeploy,Y0))*scale_ +shift_)
    # Set 0kWh as min prediction, and convert to integer
    Y_pred = np.maximum(0,int(Y_pred))
    url_response = send_value2url(Y_pred)
    print(url_response.decode())


- - - - - - - - - - 
Deployment >Testing
- - - - - - - - - - 

Latest wind forecasts:
Downloaded latest forecasts from
`https://ai4impact.org/P003/`
saved to
datasets

Historical wind forecasts:
Downloaded historical forecasts from
`https://ai4impact.org/P003/historical/`
saved to
datasets

---
Time elapsed: 0 days 00:00:10
---

Downloaded from:
`https://ai4impact.org/P003/historical/energy-ile-de-france.csv`
saved to:
datasets/energy-ile-de-france.csv
Downloading
"eCO2mix_RTE_Ile-de-France_En-cours-TR.zip" from
`https://eco2mix.rte-france.com/download/eco2mix/eCO2mix_RTE_Ile-de-France_En-cours-TR.zip`

Extracting: datasets/eCO2mix_RTE_Ile-de-France_En-cours-TR.zip
eCO2mix_RTE_Ile-de-France_En-cours-TR.xls --> as "datasets/eCO2mix_RTE_Ile-de-France_En-cours-TR.xls"
- - - - - 
File Name                                             Modified             Size
eCO2mix_RTE_Ile-de-France_En-cours-TR.xls      2020-07-20 13:31:20      1092858
- - - - - 
Deleting
datasets/eCO2mix_RTE_Ile-de-Fran

In [377]:
missing_range = None
fill_range = None # need samples from this range to fill in missing range
if Y0==None:
    missing_range = pd.date_range(energy_date_range[-1]+pd.Timedelta(hours=1),need_time(),freq='H')
    fill_range = missing_range - pd.Timedelta(hours=18)

In [378]:
print(missing_range, '\n',fill_range)

DatetimeIndex(['2020-07-20 12:00:00', '2020-07-20 13:00:00',
               '2020-07-20 14:00:00', '2020-07-20 15:00:00',
               '2020-07-20 16:00:00', '2020-07-20 17:00:00',
               '2020-07-20 18:00:00', '2020-07-20 19:00:00',
               '2020-07-20 20:00:00', '2020-07-20 21:00:00'],
              dtype='datetime64[ns]', freq='H') 
 DatetimeIndex(['2020-07-19 18:00:00', '2020-07-19 19:00:00',
               '2020-07-19 20:00:00', '2020-07-19 21:00:00',
               '2020-07-19 22:00:00', '2020-07-19 23:00:00',
               '2020-07-20 00:00:00', '2020-07-20 01:00:00',
               '2020-07-20 02:00:00', '2020-07-20 03:00:00'],
              dtype='datetime64[ns]', freq='H')


In [ ]:
# we need this range of dates for features
oldest_time = need_time()-pd.Timedelta(hours=window_size-1)
wind_speed_latest_time = need_time()+pd.Timedelta(hours=18)
wind_speed_range = pd.date_range(oldest_time,wind_speed_latest_time,freq='H')

wind_df = read_wind_forecasts_w_range(wind_speed_range)
print(f'---\nTime elapsed: {utc_now()-t0}\n---\n')

wind_norm = wind_df.values/wind_scale_
wind_speeds = np.sqrt(wind_norm[:,0::2]**2 + wind_norm[:,1::2]**2)# wind speed from wind vectors

X_norm_diffwindows = []
for l in range(wind_speeds.shape[1]):
    X_norm_diffwindows.append(
        datautils.windowed_diff_data(wind_speeds[:,l], lead_time=lead_time, window_size=window_size))

In [ ]:
# normalised data
Yt = data_norm[:,0].reshape(-1,1)

start_time = window_size-1 # index of last elem of window

# Y(t)
# prepare windows : 1st columns Y(T+lead_time)
Y_norm_diffwind = windowed_diff_data(Yt, lead_time=lead_time, window_size=window_size)
print(f'\nFull Diff windowed dataset (1st column is target Y(T+{lead_time}): {Y_norm_diffwind.shape}',
      f"; windowsize:{window_size}")